# Cython and NumPy, buffers, fused types

In [39]:
%load_ext Cython
import numpy as np

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Cython and numpy arrays -- original syntax

In [54]:
def shannon_entropy_py(p_x):
    return - np.sum(p_x * np.log(p_x))

In [41]:
%%cython

import numpy as np
cimport numpy as cnp

# https://en.wiktionary.org/wiki/Shannon_entropy

def shannon_entropy_cy(cnp.ndarray p_x):
    return - np.sum(p_x * np.log(p_x))
    

In [11]:
# entropy of the uniform distribution: log(n)

N = 1000
print(shannon_entropy_cy(np.ones((N,), dtype=np.float64) / N))
print(np.log(N))

# entropy of binomial distribution: 0.5 log(2 * pi * e * n * p * (1 - p))

from scipy.st

6.90775527898
6.90775527898


In [22]:
from scipy.stats import binom, uniform, poisson

In [28]:
n, mu = 100, 10.0
print(poisson.entropy(mu))
print(shannon_entropy_cy(poisson.pmf(np.arange(n), mu)))

2.5614099352749125
2.56140993527


In [29]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_cy(d)

The slowest run took 4.75 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.13 µs per loop


In [49]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_py(d)

The slowest run took 7.84 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.22 µs per loop


In [50]:
%%timeit poi=poisson(mu)
poi.entropy()

1000 loops, best of 3: 838 µs per loop


In [60]:
%%cython

cimport cython
cimport numpy as cnp
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
def shannon_entropy_faster(cnp.ndarray[double] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [61]:
d = poisson.pmf(np.arange(n), mu)
(shannon_entropy_py(d), shannon_entropy_faster(d))

(2.5614099352749125, 2.5614099352749107)

In [62]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_faster(d)

The slowest run took 5.46 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.78 µs per loop


In [65]:
%%cython

cimport cython
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
def shannon_entropy_mv(double[::1] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [67]:
d = poisson.pmf(np.arange(n), mu)
(shannon_entropy_py(d), shannon_entropy_mv(d))

(2.5614099352749125, 2.5614099352749107)

In [68]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_mv(d)

The slowest run took 6.33 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.76 µs per loop
